<a href="https://colab.research.google.com/github/matiasvallejosdev/my-face-recognition/blob/main/Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Preprocesing Faces 
Work reference:
[Image Pre-processing Using OpenCV Library on MORPH-II Face Database](https://uncw.edu/math/reu/documents/image-pre-processing.pdf)

In [84]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
import json
import time

from console_progressbar import ProgressBar
from time import time
from datetime import date
from google.colab import files
from google.colab.patches import cv2_imshow
from google.colab import drive

In [ ]:
!pip install console_progressbar

## 1. Prepare data

1. Initialize in memory classifiers
2. Set parameters for cascade classifiers
3. Get data images_faces from drive

### Step 1.1

In [2]:
drive.mount('/content/drive') 

Mounted at /content/drive


In [20]:
!ls "/content/drive/My Drive/Machine Learning/Datasets"

cat-non-cat  cat-vs-dog  googleplaystore_reviews  my-face


In [10]:
URL_DATA = '/content/drive/My Drive/Machine Learning'

In [17]:
# Prepare data
faces_cascade = URL_DATA + '/Utils/OpenCvData/haarcascade_frontalface_alt2.xml'
eyes_cascade = URL_DATA + '/Utils/OpenCvData/haarcascade_eye_tree_eyeglasses.xml'

In [18]:
faces_cascade = cv2.CascadeClassifier(faces_cascade)
eyes_cascade = cv2.CascadeClassifier(eyes_cascade)

print(faces_cascade)
print(eyes_cascade)
print("CascadeClassifier loaded with sucess!")

<CascadeClassifier 0x7fe35859ec10>
<CascadeClassifier 0x7fe35859e710>
CascadeClassifier loaded with sucess!


### Step 1.2

In [67]:
# Classifier Parameters
faceScale = 1.05
faceMinNeighborgs = 5

eyeScale = 1.05
eyesMinNeighborgs = 5

print("Face scale is:", faceScale)
print("Face minimal neighborgs is:", faceMinNeighborgs)

print("\nEye scale is:", eyeScale)
print("Eye minimal neighborgs is:", eyesMinNeighborgs)

Face scale is: 1.05
Face minimal neighborgs is: 5

Eye scale is: 1.05
Eye minimal neighborgs is: 5


### Step 1.3

In [55]:
# Get data and preparing
PATH = URL_DATA + '/Temp/Original/'
name_dataset = 'AgustinData'
img_names= os.listdir(PATH)

print("Preparing data from: {}".format(name_dataset))
print("Datapath from drive is:", PATH)
print("Total images collected:", len(img_names))

Preparing data from: AgustinData
Datapath from drive is: /content/drive/My Drive/Machine Learning/Temp/Original/
Total images collected: 1284



## 2. Preparing data for training a deep neural network

1. Define Functions
2. Define Model
  - Convert to grayscale
  - Face detection
  - Eye detection
  - Image Rotation
  - Face and Eye Re-detection
  - Cropping and Scaling
  - Save images
  - Track metrics
3. Execute model
3. Manual Preprocesing undetactable Images


### Step 2.1
Define functions

In [27]:
def GetCropFace(imgGray, scale, minNeig):
    grayCrop = []
    faceDetection = faces_cascade.detectMultiScale(imgGray, scale, minNeig)
    print('Can detect(face): {}'.format(len(faceDetection)))
    
    if len(faceDetection) == 1:
      (x, y, w, h) = faceDetection[0]
      grayCrop.append(imgGray[y:y+h, x:x+w])

      cv2.rectangle(imgGray, (x, y), (x+w, y+h), (0, 255, 0), 1);
      ShowImage(imgGray[y:y+h, x:x+w])

    return faceDetection, grayCrop

In [87]:
def SaveImage(img, total_facedetection, num):
    datapath_saving = URL_DATA + '/Temp'

    if total_facedetection == 1:
        # Process finished correctly    
        datapath_saving = datapath_saving + '/Correct/' + 'out_' +str(num)+ '.jpg'
        #print('Saving on: ', datapath_saving)
    elif total_facedetection > 1:
        # Multiples faces on image
        datapath_saving = datapath_saving + '/Multiple/' + 'mult_' +str(num)+ '.jpg'
        #print('Saving on: ', datapath_saving)
    else:
        # Face undetacteble
        datapath_saving = datapath_saving + '/Undetect/' + 'undetect_' +str(num)+ '.jpg'
        #print('Saving on: ', datapath_saving)

    SaveOpenCvImage(img, datapath_saving)

def SaveOpenCvImage(img, path):
    try:
        cv2.imwrite(path, img)
    except Exception as e:
        print('Exception: ', e)

In [29]:
def ShowImage(img):    
    fig= plt.figure(figsize=(5, 5))
    rows = 1

    fig.add_subplot(rows, 1, 1)
    plt.imshow(img[:,:,::-1])
    plt.show()

In [30]:
def GetCropEyes(imgGray, faceDetection):
    faceGray_crop = []
    eyesDetection = []
    result = False

    eyeDetection = eyes_cascade.detectMultiScale(faceGray, eyeScale, eyesMinNeighborgs) # Eyes detector      
    print('Eyes detection: ', len(eyeDetection))

    if len(eyeDetection) != 2 or len(eyeDetection) < 2:
        return result, eyeDetection, faceGray_crop

    for (ex, ey ,ew, eh) in eyeDetection: 
        result = True
        cv2.rectangle(faceGray, (ex,ey), (ex+ew, ey+eh), (0, 255, 0), 4)
        faceGray_crop.append(faceGray) # Facegray
        
    return result, eyeDetection, faceGray_crop

In [31]:
def image_resize(image, width = None, height = None, inter = cv2.INTER_AREA):
    # initialize the dimensions of the image to be resized and
    # grab the image size
    dim = None
    (h, w) = image.shape[:2]

    # if both the width and height are None, then return the
    # original image
    if width is None and height is None:
        return image

    # check to see if the width is None
    if width is None:
        # calculate the ratio of the height and construct the
        # dimensions
        r = height / float(h)
        dim = (int(w * r), height)

    # otherwise, the height is None
    else:
        # calculate the ratio of the width and construct the
        # dimensions
        r = width / float(w)
        dim = (width, int(h * r))

    # resize the image
    resized = cv2.resize(image, dim, interpolation = inter)

    # return the resized image
    return resized

In [105]:
def TrackMetrics(name, correct, incorrect, fail, min_i, max_i, execute_time):
  metrics = {
            "Name": name,
            "Images": (max_i - min_i),
            "Min": min_i,
            "Max": max_i,
            "CorrectProcess": correct,
            "IncorrectProcess": incorrect,
            "Accuracy": (correct / (correct + incorrect)) * 100,
            "Exceptions": fail,
            "ExecutionTime": execute_time,    
          }
          
  print(json.dumps(metrics))

  x = json.dumps(metrics)
  y = x.encode('utf-8')

  with open("(metrics){}_{}to{}_{}.json".format(name, min_i, max_i, date.today()), 'w') as f:
    f.write(json.dumps(metrics))
  files.download("(metrics){}_{}to{}_{}.json".format(name, min_i, max_i, date.today()))

### Step 2.1
Define model

In [111]:
def FaceProcessData(path, img_names, min_i, max_i, widht_image = 32, height_image = 32, reset = True, metrics = True):
  # Process face images of img_names path
  print('Initializing FaceProcessData()\n')
  print('Parameters: ')
  print('--------------------------------------')
  print('Name: {}'.format(name_dataset))
  print('Images: {}'.format(max_i - min_i))
  print('Widht out: {}'.format(widht_image))
  print('Height out: {}'.format(height_image))
  print('Reset: {}'.format(reset))
  print('Metrics: {}'.format(metrics))
  print('Saving files in: {}'.format(URL_DATA))
  print('--------------------------------------\n')

  if reset:
    fail = 0
    correct = 0
    incorrect = 0
    index = 0

  pb = ProgressBar(total=max_i, prefix='Progress', suffix='Complete', decimals=2, length=50, zfill='-')
  tiempo_inicial = time()

  for imgname in img_names[min_i : max_i]:
      index = index + 1
      pb.print_progress_bar(index)

      img_path = os.path.join(path, imgname)
      img = cv2.imread(img_path)

      #print('\nRead Image: ', imgname)
      try:
          # 1. Convert to gray scale
          gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
          # 2. Face detection
          faceDetection = faces_cascade.detectMultiScale(gray, faceScale, faceMinNeighborgs)
          #print('Can detect(face): ', len(faceDetection))


          if len(faceDetection) == 1:
            (x, y, w, h) = faceDetection[0]

            # 3. Eye detection
            #eyesDetection = eyes_cascade.detectMultiScale(cropImage, eyeScale, eyesMinNeighborgs)
            #print('Can detect(eyes): ', len(eyesDetection))

            """if len(eyesDetection) == 2:
              for (ex, ey, ew, eh) in eyesDetection:
                  cv2.rectangle(img, (x+ex, y+ey), (x+ex+ew, y+ey+eh), 
                                # (255, 255, 255), 1)"""
              
            # 4. Cropping and scaling
            cropImage = img[y:y+h, x:x+w]
            cropImage = image_resize(cropImage, 32, 32)

            # 5. Normalize image 
            norm_img = np.zeros((300, 300))
            norm_img = cv2.normalize(cropImage, norm_img, 0, 255, cv2.NORM_MINMAX)

            # 6. Save image
            #ShowImage(norm_img)
            SaveImage(norm_img, len(faceDetection), index)

            # 7. Track metrics
            correct = correct + 1
            #print("Correctly process detection!")
              
          if len(faceDetection) == 0 or len(faceDetection) > 1:
              # Save incorrect image and track metrics
              incorrect = incorrect + 1
              SaveImage(img, len(faceDetection), index)
              #print("Incorrectly process detection!") 

      except Exception as e:
          fail = fail + 1
          print('Exception:', e)

  # Track metrics
  tiempo_final = time() 
  execute_time = tiempo_final - tiempo_inicial

  print('\nExecute time: {}'.format(str(execute_time)))
  if metrics:
    TrackMetrics(name_dataset, correct, incorrect, fail, min_i, max_i, execute_time)

### Step 2.2
Execute model

In [112]:
PATH = URL_DATA + '/Temp/Original/'
min = 0
max = 1

FaceProcessData(PATH, img_names, min_i=min, max_i=max)

Initializing FaceProcessData()

Parameters: 
--------------------------------------
Name: AgustinData
Images: 1
Widht out: 32
Height out: 32
Reset: True
Metrics: True
Saving files in: /content/drive/My Drive/Machine Learning
--------------------------------------

Progress |██████████████████████████████████████████████████| 100.00% Complete

Execute time: 7.247582674026489
{"Name": "AgustinData", "Images": 1, "Min": 0, "Max": 1, "CorrectProcess": 1, "IncorrectProcess": 0, "Accuracy": 100.0, "Exceptions": 0, "ExecutionTime": 7.247582674026489}


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>